In [6]:
import os
import re

import emoji
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

# tf and keras
import tensorflow as tf
from tensorflow.keras import Sequential, layers, losses
from tensorflow.keras.layers import (
    Dense,
    Embedding,
    GlobalAveragePooling1D,
    Dropout,
    TextVectorization,
    Input,
    Conv1D,
    LSTM,
    MaxPooling1D,
    Bidirectional,
)
from tensorflow.keras.models import Model
# import tensorflow_datasets as tfds

pd.set_option('display.max_colwidth', 100) 

# UTILS

In [7]:
def read_files():
    X_train = pd.read_csv('./data/final/X_train.csv')
    y_train = pd.read_csv('./data/final/y_train.csv')
    X_val = pd.read_csv('./data/final/X_val.csv')
    y_val = pd.read_csv('./data/final/y_val.csv')
    X_test = pd.read_csv('./data/final/X_test.csv')
    y_test = pd.read_csv('./data/final/y_test.csv')
    
    train_not_na_indices = (X_train['fulltext'].notna())
    val_not_na_indices = (X_val['fulltext'].notna())
    test_not_na_indices = (X_test['fulltext'].notna())
    
    X_train = X_train[train_not_na_indices]
    X_val = X_val[val_not_na_indices]
    X_test = X_test[test_not_na_indices]
    
    y_train = y_train[train_not_na_indices]
    y_val = y_val[val_not_na_indices]
    y_test = y_test[test_not_na_indices]

    return X_train, y_train, X_val, y_val, X_test, y_test

In [8]:
def get_vectorization_layer(df, column, max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    vectorize_layer = layers.TextVectorization(
        max_tokens=max_tokens,
        output_mode='int',
        output_sequence_length=output_sequence_length)

    df[column] = df[column].astype(str)
    vectorize_layer.adapt(df[column].values)

    return vectorize_layer

In [9]:
X_train, y_train, X_val, y_val, X_test, y_test = read_files()

### Convert the fulltext into tensors

In [10]:
text_data = tf.constant(X_train['fulltext'].values)
text_data_val = tf.constant(X_val['fulltext'].values)
text_data_test = tf.constant(X_test['fulltext'].values)

2024-07-30 20:26:23.830909: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


### Get vectorization layer

In [11]:
vectorize_layer = get_vectorization_layer(X_train, 'fulltext')

### Vectorize Text

In [12]:
vectorized_text = vectorize_layer(text_data)
vectorized_text_val = vectorize_layer(text_data_val)
vectorized_text_test = vectorize_layer(text_data_test)

# Model 1 - Deleted

# Model 2: TextVectorization Layer, Basic Embedding Model, Two Hidden Dense Layers (64, 32)

In [13]:
def build_model_2(max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))
    
    x = Embedding(input_dim=max_tokens, output_dim=embedding_dim, input_length=output_sequence_length)(inputs)

    x = GlobalAveragePooling1D()(x)

    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
    
    return model

In [14]:
model_2 = build_model_2()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_2 = model_2.fit(vectorized_text, y_train, epochs=5, batch_size=2,verbose=1, callbacks=[early_stopping], validation_data=(vectorized_text_val, y_val))
hist2 = pd.DataFrame(history_2.history)
hist2

Epoch 1/5
14787/14787 [==============================] - 361s 24ms/step - loss: 0.9407 - mean_absolute_error: 0.1118 - accuracy: 0.0000e+00 - val_loss: 0.6097 - val_mean_absolute_error: 0.1048 - val_accuracy: 0.0000e+00
Epoch 2/5
14787/14787 [==============================] - 301s 20ms/step - loss: 0.8604 - mean_absolute_error: 0.1165 - accuracy: 0.0000e+00 - val_loss: 0.6275 - val_mean_absolute_error: 0.1179 - val_accuracy: 0.0000e+00


,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,0.940697,0.111765,0.0,0.609747,0.104846,0.0
1,0.860433,0.116496,0.0,0.627505,0.117921,0.0


In [15]:
model_2.save("model2.h5")

In [19]:
hist2.to_csv('hist2.csv')

# Model 3: TextVectorization Layer, Convolutional NN Embedding Model, Two Hidden Dense Layers (64, 32)

In [16]:
def build_model_3(max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))
    
    x = Embedding(input_dim=max_tokens, output_dim=embedding_dim, input_length=output_sequence_length)(inputs)

    x = layers.Conv1D(32, 4, activation='relu')(x)
    x = layers.MaxPooling1D()(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)

    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
    
    return model

In [20]:
model_3 = build_model_3()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_3 = model_3.fit(vectorized_text, y_train, epochs=5, batch_size=2, verbose=1, callbacks=[early_stopping], validation_data=(vectorized_text_val, y_val))
hist_3 = pd.DataFrame(history_3.history)
hist_3

Epoch 1/5
14787/14787 [==============================] - 468s 32ms/step - loss: 0.9595 - mean_absolute_error: 0.1176 - accuracy: 0.0000e+00 - val_loss: 0.6870 - val_mean_absolute_error: 0.1062 - val_accuracy: 0.0000e+00
Epoch 2/5
14787/14787 [==============================] - 414s 28ms/step - loss: 0.9428 - mean_absolute_error: 0.1188 - accuracy: 0.0000e+00 - val_loss: 0.6797 - val_mean_absolute_error: 0.1011 - val_accuracy: 0.0000e+00
Epoch 3/5
14787/14787 [==============================] - 474s 32ms/step - loss: 0.9323 - mean_absolute_error: 0.1239 - accuracy: 0.0000e+00 - val_loss: 0.6901 - val_mean_absolute_error: 0.1014 - val_accuracy: 0.0000e+00


,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,0.959536,0.117650,0.0,0.686980,0.106175,0.0
1,0.942791,0.118814,0.0,0.679683,0.101125,0.0
2,0.932319,0.123929,0.0,0.690050,0.101403,0.0


In [21]:
model_3.save("model3.h5")

In [22]:
hist_3.to_csv('hist_3.csv')

In [27]:
def build_model_3_b(max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))
    
    x = Embedding(input_dim=max_tokens, output_dim=embedding_dim, input_length=output_sequence_length)(inputs)

    x = layers.Conv1D(32, 4, activation='relu')(x)
    x = layers.MaxPooling1D()(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)

    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
    
    return model

In [28]:
model_3_b = build_model_3_b()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_3_b = model_3_b.fit(vectorized_text, y_train, epochs=5, batch_size=2, verbose=1, callbacks=[early_stopping], validation_data=(vectorized_text_val, y_val))
hist_3_b = pd.DataFrame(history_3_b.history)
hist_3_b

Epoch 1/5
14787/14787 [==============================] - 506s 34ms/step - loss: 0.9680 - mean_absolute_error: 0.1181 - accuracy: 0.0000e+00 - val_loss: 0.7059 - val_mean_absolute_error: 0.1227 - val_accuracy: 0.0000e+00
Epoch 2/5
14787/14787 [==============================] - 487s 33ms/step - loss: 0.9622 - mean_absolute_error: 0.1253 - accuracy: 0.0000e+00 - val_loss: 0.6816 - val_mean_absolute_error: 0.0962 - val_accuracy: 0.0000e+00
Epoch 3/5
14787/14787 [==============================] - 476s 32ms/step - loss: 0.9510 - mean_absolute_error: 0.1224 - accuracy: 0.0000e+00 - val_loss: 0.7079 - val_mean_absolute_error: 0.0995 - val_accuracy: 0.0000e+00


,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,0.968016,0.118088,0.0,0.705901,0.122682,0.0
1,0.962250,0.125279,0.0,0.681570,0.096180,0.0
2,0.950995,0.122447,0.0,0.707882,0.099532,0.0


In [30]:
model_3_b.save("model3_b.h5")
hist_3_b.to_csv('hist_3_b.csv')

In [23]:
def build_model_3_dropout(max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))
    
    x = Embedding(input_dim=max_tokens, output_dim=embedding_dim, input_length=output_sequence_length)(inputs)

    x = layers.Conv1D(32, 4, activation='relu')(x)
    x = layers.MaxPooling1D()(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
    
    return model

In [24]:
model_3_dp = build_model_3_dropout()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_3_dp = model_3_dp.fit(vectorized_text, y_train, epochs=5, batch_size=2, verbose=1, callbacks=[early_stopping], validation_data=(vectorized_text_val, y_val))
hist_3_dp = pd.DataFrame(history_3_dp.history)
hist_3_dp

Epoch 1/5
14787/14787 [==============================] - 407s 27ms/step - loss: 0.9607 - mean_absolute_error: 0.1231 - accuracy: 0.0000e+00 - val_loss: 0.6941 - val_mean_absolute_error: 0.1262 - val_accuracy: 0.0000e+00
Epoch 2/5
14787/14787 [==============================] - 408s 28ms/step - loss: 0.9477 - mean_absolute_error: 0.1196 - accuracy: 0.0000e+00 - val_loss: 0.6791 - val_mean_absolute_error: 0.1381 - val_accuracy: 0.0000e+00
Epoch 3/5
14787/14787 [==============================] - 424s 29ms/step - loss: 0.9393 - mean_absolute_error: 0.1213 - accuracy: 0.0000e+00 - val_loss: 0.6873 - val_mean_absolute_error: 0.1142 - val_accuracy: 0.0000e+00


,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,0.960744,0.123068,0.0,0.694064,0.126233,0.0
1,0.947665,0.119645,0.0,0.679076,0.138082,0.0
2,0.939346,0.121279,0.0,0.687265,0.114197,0.0


In [25]:
model_3_dp.save("model3_dp.h5")

In [26]:
hist_3_dp.to_csv('hist_3_dp.csv')

# Model 4: TextVectorization Layer, LSTM RNN Embedding Model, Two Hidden Dense Layers (64, 32)

In [31]:
def build_model_4_tanh(max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))
    
    x = Embedding(input_dim=max_tokens, output_dim=embedding_dim, input_length=output_sequence_length)(inputs)
    
    x = layers.LSTM(32, activation='tanh')(x)
    # x = layers.MaxPooling1D()(x)
    
    x = Dense(64, activation='tanh')(x)
    x = Dense(32, activation='tanh')(x)

    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])

    return model

In [32]:
model_4_tanh = build_model_4_tanh()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_4_tanh = model_4_tanh.fit(vectorized_text, y_train, epochs=5, batch_size=2, verbose=1, callbacks=[early_stopping], validation_data=(vectorized_text_val, y_val))
hist_4_tanh = pd.DataFrame(history_4_tanh.history)

2024-07-31 07:58:52.330468: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-31 07:58:52.336520: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-31 07:58:52.340800: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/5


2024-07-31 07:58:55.313395: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-31 07:58:55.321563: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-31 07:58:55.329862: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

14787/14787 [==============================] - ETA: 0s - loss: 0.9682 - mean_absolute_error: 0.1216 - accuracy: 0.0000e+00

2024-07-31 09:04:56.422212: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-31 09:04:56.428072: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-31 09:04:56.432751: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

14787/14787 [==============================] - 4166s 281ms/step - loss: 0.9682 - mean_absolute_error: 0.1216 - accuracy: 0.0000e+00 - val_loss: 0.7066 - val_mean_absolute_error: 0.1193 - val_accuracy: 0.0000e+00
Epoch 2/5
14787/14787 [==============================] - 3562s 241ms/step - loss: 0.9682 - mean_absolute_error: 0.1244 - accuracy: 0.0000e+00 - val_loss: 0.7088 - val_mean_absolute_error: 0.1005 - val_accuracy: 0.0000e+00


In [35]:
hist_4_tanh

,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,0.968242,0.121550,0.0,0.706560,0.119283,0.0
1,0.968232,0.124366,0.0,0.708794,0.100467,0.0


In [33]:
model_4_tanh.save("model4_tanh.h5")

In [34]:
hist_4_tanh.to_csv('hist_4_tanh.csv')

# Model 5: TextVectorization Layer, Bi-Directional LSTM RNN Embedding Model, Two Hidden Dense Layers (64, 32)

In [36]:
def build_model_5_tanh(max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))
    x = Embedding(input_dim=max_tokens, output_dim=embedding_dim, input_length=output_sequence_length)(inputs)
    
    x = Bidirectional(LSTM(32, activation='tanh'))(x)
    # x = layers.MaxPooling1D()(x)
    
    x = Dense(64, activation='tanh')(x)
    x = Dense(32, activation='tanh')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
    
    return model

In [37]:
model_5_tanh = build_model_5_tanh()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_5_tanh = model_5_tanh.fit(vectorized_text, y_train, epochs=5, batch_size=2, verbose=1, callbacks=[early_stopping], validation_data=(vectorized_text_val, y_val))
hist_5_tanh = pd.DataFrame(history_5_tanh.history)
hist_5_tanh

2024-07-31 10:07:41.764336: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-31 10:07:41.766802: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-31 10:07:41.768389: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/5


2024-07-31 10:07:41.981980: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-31 10:07:41.986243: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-31 10:07:41.988697: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

14787/14787 [==============================] - ETA: 0s - loss: 0.9673 - mean_absolute_error: 0.1585 - accuracy: 0.0000e+00

2024-07-31 11:33:56.485077: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-31 11:33:56.488362: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-31 11:33:56.491713: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

14787/14787 [==============================] - 5497s 371ms/step - loss: 0.9673 - mean_absolute_error: 0.1585 - accuracy: 0.0000e+00 - val_loss: 0.7064 - val_mean_absolute_error: 0.1267 - val_accuracy: 0.0000e+00
Epoch 2/5
14787/14787 [==============================] - 5459s 369ms/step - loss: 0.9297 - mean_absolute_error: 0.1337 - accuracy: 0.0000e+00 - val_loss: 0.6219 - val_mean_absolute_error: 0.1132 - val_accuracy: 0.0000e+00
Epoch 3/5
14787/14787 [==============================] - 5682s 384ms/step - loss: 0.8856 - mean_absolute_error: 0.1483 - accuracy: 0.0000e+00 - val_loss: 0.6408 - val_mean_absolute_error: 0.1067 - val_accuracy: 0.0000e+00


,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,0.967287,0.158545,0.0,0.706388,0.126739,0.0
1,0.929697,0.133679,0.0,0.621943,0.113237,0.0
2,0.885640,0.148274,0.0,0.640811,0.106668,0.0


In [39]:
model_5_tanh.save("model5_tanh.h5")

In [38]:
hist_5_tanh.to_csv('hist_5_tanh.csv')

# Model Evaluations

In [41]:
hist2

,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,0.940697,0.111765,0.0,0.609747,0.104846,0.0
1,0.860433,0.116496,0.0,0.627505,0.117921,0.0


In [42]:
hist_3

,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,0.959536,0.117650,0.0,0.686980,0.106175,0.0
1,0.942791,0.118814,0.0,0.679683,0.101125,0.0
2,0.932319,0.123929,0.0,0.690050,0.101403,0.0


In [43]:
hist_3_b

,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,0.968016,0.118088,0.0,0.705901,0.122682,0.0
1,0.962250,0.125279,0.0,0.681570,0.096180,0.0
2,0.950995,0.122447,0.0,0.707882,0.099532,0.0


In [44]:
hist_3_dp

,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,0.960744,0.123068,0.0,0.694064,0.126233,0.0
1,0.947665,0.119645,0.0,0.679076,0.138082,0.0
2,0.939346,0.121279,0.0,0.687265,0.114197,0.0


In [45]:
hist_4_tanh

,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,0.968242,0.121550,0.0,0.706560,0.119283,0.0
1,0.968232,0.124366,0.0,0.708794,0.100467,0.0


In [46]:
hist_5_tanh

,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,0.967287,0.158545,0.0,0.706388,0.126739,0.0
1,0.929697,0.133679,0.0,0.621943,0.113237,0.0
2,0.885640,0.148274,0.0,0.640811,0.106668,0.0


In [49]:
def validate_model(model, x_train, y_train, x_val, y_val, x_test, y_test):
    train_loss_mse, train_mae, train_acc = model.evaluate(x_train, y_train, verbose=0)
    val_loss_mse, val_mae, val_acc = model.evaluate(x_val, y_val, verbose=0)
    test_loss_mse, test_mae, test_acc = model.evaluate(x_test, y_test, verbose=0)
    
    return {
        "train": {"mse":train_loss_mse,"mae":train_mae,"acc":train_acc},
        "val": {"mse":val_loss_mse,"mae":val_mae,"acc":val_acc},
        "test": {"mse":test_loss_mse,"mae":test_mae,"acc":test_acc},
    }

In [50]:
model_2_saved = tf.keras.models.load_model('model2.h5')
model_3_saved = tf.keras.models.load_model('model3.h5')
model_3_b_saved = tf.keras.models.load_model('model3_b.h5')
model_3_dropout_saved = tf.keras.models.load_model('model3_dp.h5')
model_4_tanh_saved = tf.keras.models.load_model('model4_tanh.h5')
model_5_tanh_saved = tf.keras.models.load_model('model5_tanh.h5')

2024-07-31 17:25:32.804751: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-31 17:25:32.807242: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-31 17:25:32.811232: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [51]:
model_2_eval = validate_model(model_2_saved ,vectorized_text, y_train, vectorized_text_val, y_val, vectorized_text_test, y_test)
model_3_eval = validate_model(model_3_saved, vectorized_text, y_train, vectorized_text_val, y_val, vectorized_text_test, y_test)
model_3__b_val = validate_model(model_3_b_saved, vectorized_text, y_train, vectorized_text_val, y_val, vectorized_text_test, y_test)
model_3_dropout_eval = validate_model(model_3_dropout_saved,vectorized_text, y_train, vectorized_text_val, y_val, vectorized_text_test, y_test)
model_4_tanh_eval = validate_model(model_4_tanh_saved, vectorized_text, y_train, vectorized_text_val, y_val, vectorized_text_test, y_test)
model_5_tanh_eval = validate_model(model_5_tanh_saved, vectorized_text, y_train, vectorized_text_val, y_val, vectorized_text_test, y_test)

2024-07-31 17:27:26.126366: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-31 17:27:26.128360: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-31 17:27:26.131305: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [52]:
pd.DataFrame(model_2_eval).T

,mse,mae,acc
train,0.860489,0.098758,0.0
val,0.609747,0.104846,0.0
test,0.284968,0.083762,0.0


In [53]:
pd.DataFrame(model_3_eval).T

,mse,mae,acc
train,0.936278,0.088931,0.0
val,0.681571,0.096180,0.0
test,0.351257,0.070521,0.0


In [54]:
pd.DataFrame(model_3__b_val).T

,mse,mae,acc
train,0.936278,0.088931,0.0
val,0.681571,0.096180,0.0
test,0.351257,0.070521,0.0


In [55]:
pd.DataFrame(model_3_dropout_eval).T

,mse,mae,acc
train,0.930402,0.131928,0.0
val,0.679076,0.138082,0.0
test,0.350863,0.113639,0.0


In [56]:
pd.DataFrame(model_4_tanh_eval).T

,mse,mae,acc
train,0.967334,0.111757,0.0
val,0.706559,0.119283,0.0
test,0.378461,0.092595,0.0


In [57]:
pd.DataFrame(model_5_tanh_eval).T

,mse,mae,acc
train,0.870323,0.103918,0.0
val,0.621942,0.113237,0.0
test,0.320861,0.091160,0.0


Model 2 performed the best in terms of validation dataset loss, which calculated as the mean squared error.

Model 2 - Validation Loss after 5 Epochs - 2145500.25
Model 3 - Validation Loss after 3 Epochs - 2365950.75
Model 4 - Validation Loss after 4 Epochs - 2463587.00
Model 5 - Validation Loss after 5 Epochs - 2465304.75

In [52]:
display(test.layers)
display(test.summary())

# Retrieve the embeddings layer, which itself is wrapped in a list.
embeddings = test.layers[1].get_weights()[0]
print('-'*100)
display("Embeddings layer - shape: ", embeddings.shape)
print('-'*100)
display("Embeddings layer - parameter matrix (before training): ", embeddings)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 250)]             0         
                                                                 
 embedding (Embedding)       (None, 250, 16)           160000    
                                                                 
 bidirectional (Bidirectiona  (None, 64)               12544     
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                             

None

----------------------------------------------------------------------------------------------------


'Embeddings layer - shape: '

(10000, 16)

----------------------------------------------------------------------------------------------------


'Embeddings layer - parameter matrix (before training): '

array([[ 0.07392167, -0.01201833,  0.07342985, ...,  0.05041577,
         0.08608239,  0.05633584],
       [ 0.01863779, -0.01785451, -0.0067727 , ..., -0.02176556,
        -0.01128479,  0.01053058],
       [ 0.01540924, -0.04836084, -0.0321469 , ...,  0.01791335,
        -0.00249856, -0.00868337],
       ...,
       [ 0.0019884 ,  0.01765646, -0.01861403, ...,  0.01839873,
        -0.00617508, -0.02580006],
       [-0.01264708, -0.01301428, -0.02354966, ...,  0.02202214,
         0.01753282,  0.00889954],
       [ 0.01021462,  0.01071981,  0.04415278, ...,  0.00548404,
         0.01263346, -0.02413355]], dtype=float32)

# End of file

Sources:
* https://stackoverflow.com/questions/73878049/how-do-you-convert-the-pandas-dataframe-to-tensorflow-python-data-ops-dataset-op